In [1]:
%load_ext line_profiler
%pylab inline
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

from __future__ import print_function
import cProfile

from JSAnimation import IPython_display
from matplotlib import animation
import time
import math

from species import *
from channels import *
from compartment import *
from membrane import *
from params import *

from csdmodel1d import *

import numpy as np
np.seterr(over ='warn',under='ignore')
import scipy as sp
from scipy.integrate import ode

#from dolfin import *  # FEM library
from ProgressBar import *

import logging
logging.getLogger("FFC").setLevel(logging.ERROR)

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = CSDModelInterval(N=32,dx=50e-6) # define the model, grid spacing is 100 microns, or approximately two cell widths

# Define the compartments, and the membranes
ecs = Compartment("ecs")
neuron = CellCompartment("neuron",density = 2e5) # 2e5 neurons per meter, 4e10 per sq meter
glia = CellCompartment("glia",density = 2e5) #2e5 glia per meter

neuronal_er = CellCompartment("neuron_er",density = 2e5)
glial_er = CellCompartment("glia_er",density = 2e5)

neuronal_mito = CellCompartment("neuron_mito",density = 2e5)
glial_mito = CellCompartment("glial_mito",density = 2e5)

neuron_mem = Membrane("neuronal",inside=neuron,outside=ecs,Cm=Cn,phi_m=-70e-3)
glial_mem = Membrane("glial",inside=glia,outside=ecs,Cm=Cg,phi_m=-85e-3)

neuronal_er_mem = Membrane("neuronal_er",inside=neuronal_er,outside=neuron,Cm=Cn,phi_m=0.0)
neuronal_er_mem = Membrane("glial_er",inside=glial_er,outside=glia,Cm=Cn,phi_m=0.0)


# Add the compartments to the model
model.addCompartment(ecs,fraction=0.2) # ECS take 20% of the total volume
model.addCompartment(neuron,fraction=0.4-0.04) # Neurons take up 40% of the total volume
model.addCompartment(glia,fraction=0.4-0.04) # Neurons take up 40% of the total volume
model.addCompartment(neuronal_er,fraction = 0.02)
model.addCompartment(glial_er,fraction = 0.02)
model.addCompartment(neuronal_mito,fraction = 0.02)
model.addCompartment(glial_mito,fraction = 0.02)

# Add ion species
ecs.addSpecies(K,Ke0,name='K_e')
ecs.addSpecies(Cl,Cle0,name='Cl_e')
ecs.addSpecies(Na,Nae0,name='Na_e')
ecs.addSpecies(Ca,Cae0,name='Ca_e')
ecs.addSpecies(Glu,ge0,name = "g_e") # 4 micromolar in ecs

neuron.addSpecies(K,Ki0,0,'K_n')
neuron.addSpecies(Na,Nai0,0,'Na_n')
neuron.addSpecies(Cl,Cli0,0,'Cl_n')
neuron.addSpecies(Ca,Cai0,0,'Ca_n')
neuron.addSpecies(Glu,1e-6,name = "g_n")

glia.addSpecies(K,Kg0,name='K_g')
glia.addSpecies(Na,Nag0,name='Na_g')
glia.addSpecies(Cl,Clg0,name='Cl_g')
glia.addSpecies(Ca,Cag0,0,'Ca_g')

# add channels
neuron_mem.addChannel(NaTChannel(quasi_steady=True),10000.) # 10000 per neuron?
neuron_mem.addChannel(NaPChannel(quasi_steady=True),100.) # 100 per neuron
neuron_mem.addChannel(KDRChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(KAChannel(quasi_steady=True),10000.) # number of channels per neuron
neuron_mem.addChannel(SKChannel(),10000.) # SK
neuron_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(CaNChannel(quasi_steady=True),10000.) # number of channels per neuron
neuron_mem.addChannel(gNMDAChannel(),10000.)

neuron_mem.addChannel(PMCAPump(),10000) # PMCA pump
neuron_mem.addChannel(NaCaExchangePump(),1000) # sodium-calcium exchanger
neuron_mem.addChannel(NaKATPasePump(),4e5) # 5000 ATPase per neuron
neuron_mem.addChannel(NonSpecificChlorideChannel(phi0),100000)
neuron_mem.addChannel(AquaPorin(),1e-6) # Add water exchange

glial_mem.addChannel(KIRChannel(),50000) # KIR Channel
glial_mem.addChannel(NaKATPasePump(),4e5) # 10000000 ATPase per glia
glial_mem.addChannel(KDRglialChannel(),17500)
glial_mem.addChannel(PMCAPump(),10000)
glial_mem.addChannel(NaCaExchangePump(),1000) # sodium-calcium exchanger
glial_mem.addChannel(NonSpecificChlorideChannel(phig0),100000)
glial_mem.addChannel(AquaPorin(),1e-6) # Add water exchange

glial_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
glial_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron
#glial_mem.addChannel(CaNChannel(),10000.) # number of channels per neuron

# add glutamate exocytosis
#glutamate_exo = GlutmateExocytosis("G_exo", 1000)
#neuron_mem.addReaction(glutamate_exo)



In [3]:
model.addMembrane(neuron_mem)
model.addMembrane(glial_mem)

#model.addReaction(glutamate_exo)

neuron_mem.addLeakChannels()
neuron.balanceWith(ecs)
glial_mem.addLeakChannels()
glia.balanceWith(ecs)

model.assembleSystem()

system_state = model.getInternalVars()

Ion: Ca2+, P_leak: 4.90E-11
Ion: Na+, P_leak: 5.49E-09
Ion: K+, P_leak: 8.14E-08
Ion: Ca2+, P_leak: 6.97E-11
Ion: K+, P_leak: 1.56E-06
Ion: Na+, P_leak: 2.05E-08


In [4]:
y = model.getInternalVars()
#%timeit model.ode_rhs(0,y)

In [5]:
cProfile.run('model.ode_rhs(0,y)')

         820 function calls in 0.006 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.006    0.006 <string>:1(<module>)
       16    0.000    0.000    0.000    0.000 _methods.py:31(_sum)
        6    0.000    0.000    0.000    0.000 channel.py:112(get_h)
       14    0.000    0.000    0.000    0.000 channel.py:128(get_m)
       11    0.000    0.000    0.001    0.000 channel.py:143(permeability)
        7    0.000    0.000    0.000    0.000 channel.py:150(<dictcomp>)
        4    0.000    0.000    0.000    0.000 channel.py:152(permeability_infty)
        1    0.000    0.000    0.000    0.000 channel.py:16(scalar_mult_dict)
        4    0.000    0.000    0.000    0.000 channel.py:162(<dictcomp>)
        1    0.000    0.000    0.000    0.000 channel.py:17(<dictcomp>)
        7    0.000    0.000    0.000    0.000 channel.py:195(mdot)
        3    0.000    0.000    0.000    0.000 channel.py:203(hd

In [6]:
cProfile.run('neuron_mem.phi(y)')

         3 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 membrane.py:245(phi)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [7]:
graphviz = GraphvizOutput()
graphviz.output_file = 'ode_rhs.pdf'
graphviz.output_type = 'pdf'
with PyCallGraph(output=graphviz):
    model.ode_rhs(0,y)

In [8]:
?GraphvizOutput

In [9]:
graphviz2 = GraphvizOutput()
graphviz2.output_file = 'currents.pdf'
graphviz2.output_type = 'pdf'
with PyCallGraph(output=graphviz2):
    neuron_mem.currents(y)

In [10]:
val = ecs.value(K)
val2 = np.zeros(len(val))

In [15]:
%timeit model.ode_rhs(0,y)

100 loops, best of 3: 4.6 ms per loop


In [12]:
ecs.diffusivities[K]

1.96e-09

In [13]:
%timeit id(K)

10000000 loops, best of 3: 93.9 ns per loop


In [14]:
id(Na)

139658038929808